## Useful functions and objects for computation

### Folder Creation to save Output 

In [ ]:
function makeFolder(dir::AbstractString)  # Create the input directory if it doesnt exist 
    if !isdir(dir)
        mkpath(dir)
    end
end 

### Cell definition

In [ ]:
type Cell 
    x::Float64  # Position
    state::AbstractString  # Dynamic state: "S"=static, "CW"=clockwise, "CCW"=counter clockwise
    v::Float64  # Velocity
    fTot::Float64;  # Sum of forces acting on cell except diffusion
    fC::Float64; #contact forces
    xTrue::Float64; # Position without the periodic boundaries arrangement. Used for a few calculations, like mean velocity. 
    # Constructors: 
    Cell(x::Float64, state::AbstractString) = new(x,state,0.0,0.0,0.0,x)
    Cell(x::Float64) = new(x,"S",0.0,0.0,0.0,x)
end

### Function for contact force

In [ ]:
# This function returns y-x if x<=y, 0 otherwise; used to calculate the contact force between two adjacent cells

function negPresence(x::Float64, y::Float64)
    if x<=y
        return (y-x);
    else
        return 0.0;
    end
end

## Main

### Parameters

In [ ]:
M = 1::Int; # number of simulations to run

L = 60.0::Float64; # length of the periodic-boundaries stripe (in 10^-5 m)
R = 2.0::Float64;   # radius of cells (in 10^-5 m)
nCell = 7::Int;   # number of cells 

mu = 5.0e10::Float64; # friction coefficient. 
fMot = 8.3e-9*mu::Float64;  # motile force.
fCont = 5.5*fMot::Float64;  # contact force maximum.
sigma = 4.5e-8*mu::Float64; # standard deviation of Langevin gaussian noise.  

f0 = 100.0::Float64;

nTimeStep = 600::Int;  # number of time steps.
dt = 0.01;  # numerical time step size. Real time step size = dt*Tm with Tm = muR/fMot. Use 0.01.
tInt = 1.0*60;  # Integration time for probability function. 

directedRot = false;# If true, all cells are polarized in same direction. 
randomPol = false;  # If true, all cells are randomly polarized. 

#  Integer scientific notation of mu for folder name. 
muA = mu; muB = 0;
while muA>=100
    muA /=10;
    muB+=1;
end

folder = "D:\\FolderName"::AbstractString;  # path for creating directory
folder = string(folder, "\\t$(nTimeStep)_dt$(dt)_tInt$(tInt)\\L$(Int(round(L)))_R$(Int(round(R)))\\nCell$(nCell)\\fMot$(Int(round(fMot)))_fCont$(Int(round(fCont)))_sigma$(Int(round(sigma)))_mu$(Int(round(muA)))e$(muB)\\f0_$(Int(round(f0)))")

In [ ]:
tC = mu*R*10.0^(-5)/fMot;  # Characteristic time. We chose motile mov time. 
xC = R; # Characteristic length. The real characteristic length is R*10-5 so it appears like that in dGamma and tC.
dGamma = mu*R*10.0^(-5)/tC;  # Dimension-remover factor. v* = sum(F)/dGamma. If tC = motile mov time, = fMot. 

using Distributions
gNoise = Normal(0.0,sigma/sqrt(dt*tC)); # Diffusive position noise distribution.

### Main code 

In [ ]:
posArray=Float64[];  # Position array. Filled at each t to calculate the neighbours of each cell for contact F.
forceArray = zeros(nCell,nTimeStep+Int(ceil(tInt/(dt*tC))));  # Used for integrating forces. 

#############################

for m=1:M
    noiseArray = rand(gNoise,nCell,nTimeStep);  # random sample from gN distribution. Diffusive noise. 

                    #### Create Directory 
    dir = string(folder,"\\exp$(dec(m,2))");
    makeFolder(dir);

                    #### Initialize the cell positions and states
    cellPop = Array{Cell}(nCell,1); 
    using StatsBase 
    if nCell < ceil(L/(2*R)) 
        for i=1:nCell
            
            cellPop[i,1] = Cell(Float64(L*i/(nCell*xC)));

            if directedRot
                cellPop[i,1].state = "CCW";
                
            elseif randomPol
                pPol = rand(Float64);
                if pPol < 1.0/3
                    cellPop[i,1].state = "CCW";
                elseif pPol < 2.0/3
                    cellPop[i,1].state = "CW";
                else
                    cellPop[i,1].state = "S";
                end
            end
        end
    
    else   
        for i=1:nCell
            
            cellPop[i,1] = Cell((2*i-1)*L/(2*nCell*xC));
            if directedRot
                cellPop[i,1].state = "CCW";
                
            elseif randomPol
                pPol = rand(Float64);
                if pPol < 1.0/3
                    cellPop[i,1].state = "CCW";
                elseif pPol < 2.0/3
                    cellPop[i,1].state = "CW";
                else
                    cellPop[i,1].state = "S";
                end
            end
        end
    end
                    #### Simulation
    
    for t=1:nTimeStep
        
        oFileName = "DataStep_$(dec(t,4)).csv";
        oFilePath = string(dir,"\\",oFileName);  # Path for saving output csv file.
        oArray = Array{Float64}(nCell,7);    # Output. Col1: cell number, Col2: x*, Col3: v*, Col4: fTot, Col5: state, Col6: actual x, Col7: fC. At t-1
        posArray = Array{Float64}(nCell,1);  # positions at t ie oArray[:,2] of t+1
        
        for i=1:nCell
            
                    #### Fill output csv file for cell i 
            oArray[i,1] = i;
            oArray[i,2] = cellPop[i,1].x;
            oArray[i,6] = cellPop[i,1].xTrue;
            oArray[i,3] = cellPop[i,1].v;
            if t == 1
                oArray[i,4] = dGamma*cellPop[i,1].v;
                oArray[i,7] = 0.0;
            else 
                oArray[i,4] = cellPop[i,1].fTot;
                oArray[i,7] = cellPop[i,1].fC;
            end
            cellPop[i,1].fTot = 0.0; 
            if cellPop[i,1].state == "S"
                oArray[i,5] = 0.0;
            elseif cellPop[i,1].state == "CCW"
                oArray[i,5] = 1.0;
            elseif cellPop[i,1].state == "CW"
                oArray[i,5] = -1.0;
            else
                println("Error: unknown dynamic state for cell. Must be S, CW or CCW. ")
                quit()
            end
    
                    #### Update all cell positions and save them in posArray
            cellPop[i,1].x += dt*(cellPop[i,1].v); 
            cellPop[i,1].xTrue += dt*(cellPop[i,1].v);
            while cellPop[i,1].x >= L/xC   # periodic boundary conditions on position
                cellPop[i,1].x -= L/xC;
            end
            while cellPop[i,1].x < 0
                cellPop[i,1].x += L/xC;
            end
            posArray[i,1] = cellPop[i,1].x;
        
        end
        
                    #### Generate Output csv file
        writecsv(oFilePath,oArray)  
        
        for i=1:nCell         
                    #### CONTACT FORCES
            rightArray = posArray - cellPop[i,1].x;
            for l=1:length(rightArray)  # periodic boundaries
                while rightArray[l,1]<0
                    rightArray[l,1] += L/xC;
                end
            end
            rightArray[i,1] = L/xC + 1.0;
            dRight, iRight = findmin(rightArray);  # Distance for contact force from the right.
            leftArray = cellPop[i,1].x - posArray;
            
            for l=1:length(leftArray)
                while leftArray[l,1]<0
                    leftArray[l,1] += L/xC;
                end
            end
            
            leftArray[i,1] = L/xC + 1.0;
            dLeft, iLeft = findmin(leftArray);   # Distance for contact force from the right.

            fRight = negPresence(dRight,2.0);
            fLeft = negPresence(dLeft,2.0);
            fContEff = fLeft - fRight;
            fContEff *= fCont / 2.0;
            cellPop[i,1].fTot += fContEff;
            cellPop[i,1].fC = fContEff;
            
                    #### MOTILE FORCE
            if oArray[i,5] == 0.0
                cellPop[i,1].fTot += 0.0;
            elseif oArray[i,5] == 1.0
                cellPop[i,1].fTot += fMot;
            else
                cellPop[i,1].fTot -= fMot;
            end
            
                    #### VELOCITY
            cellPop[i,1].v = (cellPop[i,1].fTot + noiseArray[i,t])/dGamma; 
        
                    #### Fill forceArray 
            if t == 1
                for l=1:Int(ceil(tInt/(dt*tC)))+1
                    forceArray[i,l] = cellPop[i,1].fTot;
                end
            else
                forceArray[i,Int(ceil(tInt/(dt*tC)))+t] = cellPop[i,1].fTot;
            end
        
                    #### Calculate integral of forces for polarity change
            intForces = mean(forceArray[i,t:Int(ceil(tInt/(dt*tC)))+t]);
        
                    #### CELL POLARITY STATES
            p = rand(Float64);  # random number between 0 and 1 
            kSigmoid = -log(-1 + 1/(1-5.0e-6)^(1));   # Guarantee P(stay in same state)~1 when intForces = 0 for dt < tInt.           
            if p < 1-(1/(1+exp(kSigmoid*(intForces/f0 - 1))))^(dt*tC/tInt)
                cellPop[i,1].state = "CCW";
            elseif p < 2 - (1/(1+exp(kSigmoid*(intForces/f0 - 1))))^(dt*tC/tInt) - (1/(1+exp(-kSigmoid*(intForces/f0 + 1))))^(dt*tC/tInt)    
                cellPop[i,1].state = "CW";
            end          
        end
    end  
end
